In [1]:
%pip install transformers
%pip install torch
%pip install datasets

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import torch

ds = load_dataset("qiaojin/PubMedQA", "pqa_labeled")

# Load ClinicalBERT tokenizer and model from "medicalai/ClinicalBERT"
tokenizer = AutoTokenizer.from_pretrained("medicalai/ClinicalBERT")
model = AutoModelForSequenceClassification.from_pretrained("medicalai/ClinicalBERT", num_labels=3)  # 3 for yes/no/maybe classification


# Move model to the available device (cuda or cpu)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Load the dataset
ds = load_dataset("qiaojin/PubMedQA", "pqa_labeled")

# Preprocess the dataset
def preprocess_function(examples):
    inputs = [f"Context: {context} Question: {question}" for question, context in zip(examples['question'], examples['context'])]
    targets = examples['final_decision']

    # Expanded label mapping to handle additional cases
    label_mapping = {"yes": 0, "no": 1, "maybe": 2, "uncertain": 2}

    # Handle missing or unexpected labels gracefully
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    model_inputs["labels"] = [label_mapping.get(target.lower(), 2) for target in targets]  # Default to 'uncertain' for unknown labels

    return model_inputs

# Select a small subset of the dataset for demonstration
small_ds = ds['train'].select(range(500))
tokenized_ds = small_ds.map(preprocess_function, batched=True)

# Split the dataset into training, validation, and test sets
train_size = 400
val_size = 50
test_size = 50

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    tokenized_ds, [train_size, val_size, test_size]
)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-6, 
    num_train_epochs=5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

# Evaluate the model
metrics = trainer.evaluate(eval_dataset=test_dataset)
print("Test set evaluation:", metrics)

# Define the function to predict using the trained model (adjusted to match BioGPT code)
def predict_answer(question, context):
    input_text = f"Context: {context} Question: {question}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True, padding="max_length")
    inputs = {key: val.to(model.device) for key, val in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()

    # Map back to "yes", "no", "uncertain"
    label_mapping = {0: "yes", 1: "no", 2: "uncertain"}
    return label_mapping[predicted_class]

# Evaluate the Q&A performance
num_correct = 0

for example in test_dataset:
    question = example['question']
    context = example['context']
    true_answer = example['final_decision']
    predicted_answer = predict_answer(question, context)

    print(f"Question: {question}")
    print(f"Context: {context}")
    print(f"True Answer: {true_answer}")
    print(f"Predicted Answer: {predicted_answer}")
    print("="*80)

    if true_answer.lower() == predicted_answer:
        num_correct += 1

# Print accuracy
accuracy = num_correct / len(test_dataset)
print(f"Accuracy: {accuracy}")


c:\Users\David\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at medicalai/ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 300/300 [00:00<00:00, 2320.40 examples/s]
c:\Users\David\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
                                              

{'eval_loss': 1.0477420091629028, 'eval_runtime': 8.3917, 'eval_samples_per_second': 5.958, 'eval_steps_per_second': 5.958, 'epoch': 1.0}


                                                  
 40%|████      | 400/1000 [08:36<12:16,  1.23s/it]

{'eval_loss': 1.2501484155654907, 'eval_runtime': 8.2669, 'eval_samples_per_second': 6.048, 'eval_steps_per_second': 6.048, 'epoch': 2.0}


 50%|█████     | 500/1000 [10:40<11:39,  1.40s/it]

...

{'loss': 1.2181, 'grad_norm': 2.0600435733795166, 'learning_rate': 0.0, 'epoch': 5.0}


                                                   
100%|██████████| 1000/1000 [21:28<00:00,  1.29s/it]


{'eval_loss': 1.533676028251648, 'eval_runtime': 8.5678, 'eval_samples_per_second': 5.836, 'eval_steps_per_second': 5.836, 'epoch': 5.0}
{'train_runtime': 1288.7033, 'train_samples_per_second': 0.776, 'train_steps_per_second': 0.776, 'train_loss': 1.1514329833984376, 'epoch': 5.0}


100%|██████████| 50/50 [00:08<00:00,  6.12it/s]


Test set evaluation: {'eval_loss': 1.242011308670044, 'eval_runtime': 8.3419, 'eval_samples_per_second': 5.994, 'eval_steps_per_second': 5.994, 'epoch': 5.0}
Question: Does high blood pressure reduce the risk of chronic low back pain?
Context: {'contexts': ['Epidemiological studies have suggested inverse relationships between blood pressure and prevalence of conditions such as migraine and headache. It is not yet clear whether similar relationships can be established for back pain in particular in prospective studies.', 'Associations between blood pressure and chronic low back pain were explored in the cross-sectional HUNT 2 survey of a Norwegian county in 1995-1997, including 39,872 individuals who never used antihypertensive medication. A prospective study, comprising 17,209 initially back pain-free individuals and 5740 individuals reporting low back pain, was established by re-examinations in the HUNT 3 survey in 2006-2008. Associations were assessed by logistic regression with resp